Chargement des données finales
Création des 3-4 graphiques professionnels
Sauvegarde dans visuals/

# Analysis of romance subgenres

**Author** : Lucie Dou  
**Date** : January 2025  
**Goal** : Identify and analyze the different subgenres of romance to understand their specific characteristics.

---

## Content
1. Loading cleaned data
2. Extraction of all present genres
3. Identifying romance subgenres
4. Creation of indicators by sub-genre
5. Comparative analyses
6. Saving enriched data

In [6]:
# importing Libraries
import pandas as pd
import numpy as np
from collections import Counter

## 1. Loading cleaned data

We load the data prepared in the previous notebook.

In [7]:
# Loading cleaned-up romances
df_romance = pd.read_csv("../data/processed/romance_books_clean.csv", 
                         encoding='utf-8', 
                         sep=';')

print(f"Dataset loaded: {len(df_romance):,} romances\n")

# Quick check
print("=== DATA OVERVIEW ===")
print(df_romance.head())

Dataset loaded: 1,566 romances

=== DATA OVERVIEW ===
   Book Id                                              Title  \
0       57  A Changeling for All Seasons (Changeling Seaso...   
1       59                                 The Changeling Sea   
2       66         The Changeling (Daughters of England  #15)   
3      151                                      Anna Karenina   
4      152                                      Anna Karenina   

                                              Author average_rating  \
0  Angela Knight/Sahara Kelly/Judy Mays/Marteeka ...           3,76   
1                               Patricia A, McKillip           4,06   
2                                      Philippa Carr           3,98   
3     Leo Tolstoy/Richard Pevear/Larissa Volokhonsky           4,05   
4       Leo Tolstoy/David Magarshack/Priscilla Meyer           4,05   

         isbn       isbn13 language_code  num_pages  ratings_count  \
0  1595962808   9,7816E+12           eng        304       

## 2. Exploration of existing genres

Before filtering, let's look at all the genres present in our dataset to identify the relevant subgenres.

In [8]:
# Extract all individual genres
all_genres = []

for genres_str in df_romance['genres'].dropna():
    # Séparer par ; et par ,
    genres_list = genres_str.replace(';', ',').split(',')
    # Nettoyer (enlever espaces)
    genres_list = [g.strip() for g in genres_list if g.strip()]
    all_genres.extend(genres_list)

# Count the occurrences
genre_counts = Counter(all_genres)

print("=== TOP 20 MOST FREQUENT GENRES ===\n")
for genre, count in genre_counts.most_common(20):
    percentage = count / len(df_romance) * 100
    print(f"{genre:35} : {count:4} ({percentage:5.1f}%)")

=== TOP 20 MOST FREQUENT GENRES ===

Romance                             : 2302 (147.0%)
Fiction                             : 1427 ( 91.1%)
Fantasy                             : 1099 ( 70.2%)
Historical                          :  985 ( 62.9%)
Literature                          :  691 ( 44.1%)
Adult                               :  523 ( 33.4%)
Contemporary                        :  522 ( 33.3%)
Historical Fiction                  :  499 ( 31.9%)
Young Adult                         :  489 ( 31.2%)
Classics                            :  482 ( 30.8%)
Womens Fiction                      :  436 ( 27.8%)
Novels                              :  408 ( 26.1%)
Chick Lit                           :  370 ( 23.6%)
Mystery                             :  350 ( 22.3%)
Paranormal                          :  338 ( 21.6%)
Cultural                            :  336 ( 21.5%)
Sequential Art                      :  335 ( 21.4%)
European Literature                 :  333 ( 21.3%)
Humor                      

## 3. Identifying romance subgenres

We identify the most represented specific subgenres of romance.

In [9]:
# Filter only subgenres containing "Romance"
print("\n=== IDENTIFIED ROMANCE SUBGENRES ===\n")

romance_subgenres = [(g, c) for g, c in genre_counts.items() if 'Romance' in g]
romance_subgenres.sort(key=lambda x: x[1], reverse=True)

for genre, count in romance_subgenres:
    percentage = count / len(df_romance) * 100
    print(f"{genre:40} : {count:4} ({percentage:5.1f}%)")


=== IDENTIFIED ROMANCE SUBGENRES ===

Romance                                  : 2302 (147.0%)
Contemporary Romance                     :  231 ( 14.8%)
Historical Romance                       :  214 ( 13.7%)
Paranormal Romance                       :  136 (  8.7%)
Category Romance                         :   90 (  5.7%)
Regency Romance                          :   38 (  2.4%)
M M Romance                              :   37 (  2.4%)
M F Romance                              :   17 (  1.1%)
Fantasy Romance                          :   16 (  1.0%)
Erotic Romance                           :   15 (  1.0%)
Christian Romance                        :   13 (  0.8%)
Western Romance                          :   10 (  0.6%)
Gothic Romance                           :    9 (  0.6%)
Medieval Romance                         :    9 (  0.6%)
Planetary Romance                        :    6 (  0.4%)
Western Historical Romance               :    5 (  0.3%)
Harlequin Romance                        :    4 (

**Main subgenres identified** (>100 occurrences) :
- Contemporary Romance
- Paranormal Romance
- Historical Romance
- Erotic Romance
- Romantic Suspense
- Fantasy Romance

These 6 subgenres will be analyzed in detail.

## 4. Creation of indicators by sub-genre

We create a boolean column for each main subgenre.

In [11]:
# Define the subgenres to be analyzed
subgenres = {
    'Contemporary': 'Contemporary Romance',
    'Historical': 'Historical Romance',
    'Paranormal': 'Paranormal Romance',
    'Erotic': 'Erotic Romance',
    'Suspense': 'Romantic Suspense',
    'Fantasy': 'Fantasy Romance'
}

# Create indicator columns
for key, genre_name in subgenres.items():
    col_name = f'is_{key.lower()}'
    df_romance[col_name] = df_romance['genres'].str.contains(genre_name, case=False, na=False)
    count = df_romance[col_name].sum()
    print(f" {genre_name:25} : {count:4} livres ({count/len(df_romance)*100:5.1f}%)")

 Contemporary Romance      :  230 livres ( 14.7%)
 Historical Romance        :  163 livres ( 10.4%)
 Paranormal Romance        :  136 livres (  8.7%)
 Erotic Romance            :   15 livres (  1.0%)
 Romantic Suspense         :   89 livres (  5.7%)
 Fantasy Romance           :   16 livres (  1.0%)


**Note** : A book can belong to several subgenres (e.g., "Paranormal Historical Romance").

## 5. Comparative analyses

Let's compare the subgenres based on different criteria.

### 5.1 Overall statistics by subgenre

In [12]:
# Prepare the results
results = []

for key, genre_name in subgenres.items():
    mask = df_romance[f'is_{key.lower()}']
    count = mask.sum()
    
    if count > 0:  # Avoid division by zero
        avg_rating = df_romance[mask]['average_rating'].mean()
        avg_engagement = df_romance[mask]['ratings_count'].mean()
        avg_pages = df_romance[mask]['num_pages'].mean()
        
        results.append({
            'Subgenre': genre_name,
            'Number': count,
            'Pourcentage': f"{count/len(df_romance)*100:.1f}%",
            'Average rating': f"{avg_rating:.2f}",
            'Average reviews': f"{avg_engagement:.0f}",
            'Average pages': f"{avg_pages:.0f}"
        })

# Create and display the table
df_results = pd.DataFrame(results)
df_results = df_results.sort_values('Number', ascending=False)

print("=== COMPARISON OF SUBGENRES ===\n")
print(df_results.to_string(index=False))

TypeError: Could not convert string '3,773,783,814,114,033,83,994,153,843,964,173,963,863,414,194,184,033,883,813,63,413,414,133,673,73,684,174,063,594,063,813,923,783,953,773,643,663,763,783,793,644,1943,943,814,093,723,523,63,973,723,824,154,094,174,033,843,84,313,914,154,153,243,373,893,743,953,723,653,74,113,813,793,693,813,953,813,953,953,753,683,693,73,623,573,673,853,714,143,923,83,843,734,034,094,113,43,964,013,893,753,453,933,974,043,553,973,633,763,623,663,643,563,73,493,943,973,563,623,543,443,514,023,453,473,743,563,583,783,773,713,593,353,543,733,623,633,593,783,923,923,664,293,923,483,983,623,943,63,813,583,623,633,453,643,633,623,193,724,153,743,733,63,793,533,833,633,843,783,943,923,83,713,643,773,483,923,943,723,543,873,313,633,693,734,253,954,23,83,913,653,993,813,813,943,633,683,964,213,233,733,583,893,673,653,684,453,833,833,653,674,223,773,773,933,683,653,613,93,83,753,823,723,713,713,723,933,853,97' to numeric

### 5.2 Ranking by quality (average rating)

In [ ]:
# Sort by average rating (convert to float for sorting)
df_results_quality = df_results.copy()
df_results_quality['Note_num'] = df_results_quality['Average rating'].astype(float)
df_results_quality = df_results_quality.sort_values('Note_num', ascending=False)

print("\n=== RANKING BY AVERAGE RATING ===\n")
for i, row in enumerate(df_results_quality.itertuples(), 1):
    medal = "🥇" if i == 1 else "🥈" if i == 2 else "🥉" if i == 3 else f"{i}."
    print(f"{medal} {row._1:25} : {row._4}/5")

### 5.3 Ranking by engagement (number of reviews)

In [ ]:
# Sort by commitment
df_results_engagement = df_results.copy()
df_results_engagement['Engagement_num'] = df_results_engagement['Average Reviews'].str.replace(',', '').astype(float)
df_results_engagement = df_results_engagement.sort_values('Engagement_num', ascending=False)

print("\n=== RANKING BY RIVIEWS ===\n")
for i, row in enumerate(df_results_engagement.itertuples(), 1):
    medal = "🔥" if i == 1 else "⭐" if i == 2 else "✨" if i == 3 else f"{i}."
    print(f"{medal} {row._1:25} : {row._5} Average Reviews")

## 6. Key Insights

Summary of main findings:

In [ ]:
# Calculate some insights automatically
best_rated = df_results_quality.iloc[0]
most_engaging = df_results_engagement.iloc[0]
most_popular = df_results.iloc[0]

print("=== Key INSIGHTS ===\n")
print(f"Most published subgenre : {most_popular['Subgenre']} ({most_popular['Number']} books)")
print(f"Best average rating : {best_rated['Subgenre']} ({best_rated['Average rate']}/5)")
print(f"Stronger commitment : {most_engaging['subgenre']} ({most_engaging['Average Reviews']} reviews)")

# Compare Historical vs Contemporary
hist_rating = float(df_results[df_results['Subgenre'] == 'Historical Romance']['Average rate'].values[0])
contemp_rating = float(df_results[df_results['Subgenre'] == 'Contemporary Romance']['Average rate'].values[0])

if hist_rating > contemp_rating:
    diff = hist_rating - contemp_rating
    print(f"\n📖 Historical Romance ({hist_rating:.2f}) is rated higher than Contemporary ({contemp_rating:.2f}) of {diff:.2f} points")
else:
    diff = contemp_rating - hist_rating
    print(f"\n📖 Contemporary Romance ({contemp_rating:.2f}) is rated higher than Historical ({hist_rating:.2f}) of {diff:.2f} points")

## 7. Saving enriched data

The data with sub-genre indicators is ready for visualization

In [ ]:
# Save the dataset with the new columns
output_path = "../data/processed/romance_with_subgenres.csv"
df_romance.to_csv(output_path, index=False, sep=';', encoding='utf-8')

# Also save the statistics table
stats_path = "../data/processed/subgenres_statistics.csv"
df_results.to_csv(stats_path, index=False)

print(f"Enriched saved dataset : {output_path}")
print(f"Statistics saved : {stats_path}")
print(f"\n {len(df_romance)} romances with {len(subgenres)} subgenre indicators")

## Next steps

**Next Notebook** : `03_visualizations.ipynb`
- Distribution graphs
- Visual comparisons of quality vs. popularity
- Distribution of ratings by subgenre